**Draft Model for Mice Steep Stage Analysis**

In [1]:
import torch
import torch.nn.functional as F
from torch import optim, nn
from torchvision import transforms, datasets, models


import torch.utils.data as utils
from inputMassager import *


In [2]:
inputHandler = inputMassager()

data_filepath = inputHandler.askForInput("Blah!")
annotated_filepath = inputHandler.askForInput("Blah!")
period_size = 200

labels, eeg_samples, emg_samples, eeg_fft, emg_fft = get_labeled_data(data_filepath, annotated_filepath, period_size)

/Users/audreyostrom/CSE144/mice-sleep-analysis/inputMassager.py:285: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:276.)
  fft_tensor[index, 0] = torch.fft.fft(torch.tensor(row[col]))


In [3]:
everything = torch.cat((eeg_samples, eeg_fft), dim = 1)
print(everything.size())

torch.Size([100889, 2, 200])


In [4]:
eeg_samples = eeg_samples[1694:10000]
emg_samples = emg_samples[1694:10000]
eeg_fft = eeg_fft[1694:10000]
emg_fft = emg_fft[1694:10000]
labels = labels[1694:10000]
print(emg_samples.size())

torch.Size([8306, 1, 200])


In [5]:
#ds = torch.utils.data.TensorDataset(eeg_fft, emg_fft, labels)
ds = torch.utils.data.TensorDataset(torch.cat((eeg_samples, eeg_fft), dim = 1), torch.cat((emg_samples, emg_fft), dim = 1), labels)

train_size = int(len(ds) *.80)
val_size = len(ds) - int(len(ds) *.80)

train_dataset, val_dataset = torch.utils.data.random_split(ds, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Creating class for Model**

In [7]:
class CNN(nn.Module):
    def __init__(self, period_size, input_channels=1):
        super(CNN, self).__init__()
        self.input_channels = input_channels

        self.eeg_conv1 = nn.Conv1d(in_channels=input_channels, out_channels=period_size, kernel_size=period_size//2)
        self.eeg_conv2 = nn.Conv1d(in_channels=period_size, out_channels=period_size//4, kernel_size=8)
        self.eeg_conv3 = nn.Conv1d(in_channels=period_size // 4, out_channels=256, kernel_size=2)

        self.emg_conv1 = nn.Conv1d(in_channels=input_channels, out_channels=period_size, kernel_size=period_size//2)
        self.emg_conv2 = nn.Conv1d(in_channels=period_size, out_channels=period_size//4, kernel_size=8)
        self.emg_conv3 = nn.Conv1d(in_channels=period_size // 4, out_channels=256, kernel_size=2)



        self.fc1 = nn.Linear(2560*2, 20)
        self.fc2 = nn.Linear(10, 4)


    def forward(self, c1, c2):
        #print(x.size())
        c1 = self.eeg_conv1(c1)
        c1 = F.relu(c1)
        # print(x.size())
        c1 = F.max_pool1d(c1, kernel_size=2)
        c1 = self.eeg_conv2(c1)
        c1 = F.max_pool1d(c1, kernel_size=2)
        # print(x.size())
        c1 = self.eeg_conv3(c1)
        c1 = F.max_pool1d(c1, kernel_size=2)
        #print(x.size())

        c2 = self.eeg_conv1(c2)
        c2 = F.relu(c2)
        # print(x.size())
        c2 = F.max_pool1d(c2, kernel_size=2)
        c2 = self.eeg_conv2(c2)
        c2 = F.max_pool1d(c2, kernel_size=2)
        # print(x.size())
        c2 = self.eeg_conv3(c2)
        c2 = F.max_pool1d(c2, kernel_size=2)


        c1c2 = torch.cat((c1, c2), dim=1)

        c1c2 = c1c2.flatten(1)
        #print(x.size())
        c1c2 = self.fc1(c1c2)
        c1c2 = F.max_pool1d(c1c2, kernel_size=2)
        # print(x.size())
        c1c2 = self.fc2(c1c2)
        c1c2 = F.log_softmax(c1c2, dim=1)
#ME      
        #c1c2 = F.relu(c1c2)
        
        return c1c2

In [8]:
def train_model(epochs, model):
    model.train() # set model to training mode
    loss_fun = nn.CrossEntropyLoss() #define a loss function object

    for epoch in range(epochs):

      for batch_idx, (channel1, channel2, target) in enumerate(train_loader):
          #print(c)
          # print(target)
          channel1, channel2, target = channel1.to(device), channel2.to(device), target.to(device)
          
          optimizer.zero_grad()
          output = model(channel1, channel2) # guess we have to pass all channels instead of data?
          loss = loss_fun(output,target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(channel1), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))

In [9]:
model = CNN(period_size, 2)
model.to(device)
  
optimizer = torch.optim.SGD(model.parameters(), lr=.01, momentum=0.9)

train_model(10, model)

Train Epoch: 0 [0/6644 (0%)]	Loss: 1.744318
Train Epoch: 1 [0/6644 (0%)]	Loss: 0.402384
Train Epoch: 2 [0/6644 (0%)]	Loss: 0.375209
Train Epoch: 3 [0/6644 (0%)]	Loss: 0.335434


KeyboardInterrupt: 

In [ ]:

def evaluate_model(model, dataloader, is_test=False, confidence_level = -0.5):
  # Set model to evaluation mode
  predictions = []
  labels_list = []
  model.eval()

  #dictionary to store the accuracy of predictions by sleep stage
  acurracy_stages = {0:[], 1:[], 2:[], 3:[], 4:[]}
  stage_names = ["not sure(0): ", "rem accuracy(1): ", "non-rem accuracy(2): ", "wake accuracy(3): ", "artifact accuracy(4): "]
  

  with torch.no_grad():
    correct = 0
    loss = 0

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    for channel1, channel2, target in dataloader:
        channel1, channel2, target = channel1.to(device), channel2.to(device), target.to(device)
        outputs = model(channel1, channel2)

        loss += torch.sum(criterion(outputs, target)).item()

        pred_value, pred = outputs.data.max(1, keepdim=True) # get the index of the max log-probability                                                         
          
        for i in range(len(target)):      
          
          #if the predicted value is lower that the confidence level, set the predicted value to 0 and the target value to 0
           if pred_value[i] < confidence_level:
            pred[i] = 0
            target[i] = 0

          #append boolean indicating whether or not each prediction matched the target to the apporpriate sleep stage list
           acurracy_stages[target[i].item()].append(pred[i].eq(target[i]).item())
           predictions.append(pred[i].item())
           labels_list.append(target[i].item())
      

        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()               
                
#calculate accuracy for each stage
    i = 0
    for key in acurracy_stages.keys():

       #acurracy_stages[key] = [acurracy_stages[key].count(True), (acurracy_stages[key].count(True) + acurracy_stages[key].count(False))]
       stage_correct = str(acurracy_stages[key].count(True))
       stage_total = str(acurracy_stages[key].count(True) + acurracy_stages[key].count(False))
       stage_accuracy = str(int(100. * (int(stage_correct) / int(stage_total))))

       print(stage_names[i] + stage_correct + "/" + stage_total + " (" + stage_accuracy + "%)")
       i += 1

#calculate total accuracy and loss
    loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        "Test" if is_test else "Validation",
        loss, correct, len(dataloader.dataset),
        accuracy))

  # Set model back to training mode
  model.train()
  return labels_list, predictions

In [ ]:
#if you make the confidence level too low, you'll get an error about not being able to divede by 0,
#that just means that none of the data is being classified to not sure, keep it below -.75 to avoid this

labels_list, predictions = evaluate_model(model, val_loader, is_test=True, confidence_level= -.75)

not sure(0): 46/46 (100%)
rem accuracy(1): 0/762 (0%)
non-rem accuracy(2): 5398/5854 (92%)
wake accuracy(3): 12829/13273 (96%)
artifact accuracy(4): 0/243 (0%)

Test set: Average loss: 0.3292, Accuracy: 18273/20178 (91%)



In [ ]:
from sklearn.metrics import classification_report

target_names = ['Artifact', 'Wake', 'Non-REM', 'REM']
print('Testing Report for Logistic Regression Model for Accuracy, Precision, Recall, and F1-Score')
print('----------------------------------------------------------------------------------')
print(classification_report(labels_list, prediction, target_names=target_names))

20178


In [ ]:
print(labels_list)
print(len(predictions))

with more training data, the model almost always predicts wake becuase it's so overepresented, when i was running just 10000 periods, it was in the mid to low 90s for non-rem and rem accuracy but with all the data, it's 99% accurate for wake and in the 30s for non rem